In [37]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from control import StateSpace, forced_response, step_info
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from ipywidgets import interact, VBox, HBox, Output, HTMLMath
from IPython.display import HTML, Math, display

HTML("""
<style>
     
.widget-label, .custom-title {
    font-weight: bold !important;
    font-size: 16px !important;
}

.widget-toggle-buttons .widget-toggle-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
.widget-toggle-buttons .widget-toggle-button.selected {
    background-color: #007acc !important;
    color: white !important;
}

button.widget-toggle-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
button.widget-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}     
</style>
""")

In [38]:
class SystemSimulator:
    def __init__(self, A = None, B = None, C = None, D = None, nonlinear_func = None):
        self.sys_lin = None
        if A is not None and B is not None and C is not None and D is not None:
            self.sys_lin = StateSpace(A, B, C, D)
        self.nonlinear_func = nonlinear_func

    def simulate(self, t, x0, u_func):
        results = {}

        if self.sys_lin is not None:
            U = [u_func(tt) for tt in t]
            T_out, y_lin = forced_response(self.sys_lin, T=t, U=U, X0=x0)
            if y_lin.ndim > 1:
                results["linear"] = y_lin[0]  
            else:
                results["linear"] = y_lin
        
        if self.nonlinear_func is not None:
            sol = solve_ivp(lambda tau, x: self.nonlinear_func(tau, x, u_func), 
                            [t[0], t[-1]], x0, t_eval=t, vectorized=False)

            if sol.success and sol.t[-1] >= t[-1]:
                results["nonlinear"] = sol.y[0]
            else:
                display(Math(fr"\textbf{{Warning: Simulation stopped early: the solver could only compute up to t = {sol.t[-1]:.2f} s}},"))
                display(Math(fr"\textbf{{while the requested simulation time was T = {t[-1]:.2f} s}}"))
                display(HTML("<div style='margin:10px 0;'></div>"))
                display(Math(r"\text{This usually happens when the system state grows too fast, i.e. diverges, or becomes unstable.}"))
                display(Math(r"\text{The plot shows the valid solution up to the stopping point.}"))

                # interpolation until last point possible
                y_interp = np.interp(t, sol.t, sol.y[0], left=np.nan, right=np.nan)
                results["nonlinear"] = y_interp
        
        return results 
    
    def calculate_specs(self, y, t):
    
        info_linear = None

        if np.isnan(y).any() or np.isinf(y).any() or abs(y[-1]) > 1e6:
            return None  # no specs for unstable response
        
        if np.std(y[-100:]) > 0.05 * abs(y[-1]):  
            return None  # oscillatory, no steady specs

        if self.sys_lin is not None:
            y = np.asarray(y).squeeze()
            t = np.asarray(t)

            info_linear = step_info(self.sys_lin, t, y)
            y_ss = float(info_linear["SteadyStateValue"])
            y0   = float(y[0])
            A    = y_ss - y0
            rising = A >= 0

            # 10% / 90% levels relative to y0 -> y_ss
            y_10 = y0 + 0.10 * A
            y_90 = y0 + 0.90 * A

            t_10 = None
            t_90 = None
            for i in range(len(y)):
                if t_10 is None and ((y[i] >= y_10) if rising else (y[i] <= y_10)):
                    t_10 = float(t[i])
                if t_90 is None and ((y[i] >= y_90) if rising else (y[i] <= y_90)):
                    t_90 = float(t[i])
                if t_10 is not None and t_90 is not None:
                    break

            info_linear["t_10"] = t_10
            info_linear["t_90"] = t_90

        return info_linear


    def plot(self, t, results, show_specs = False):
        plt.figure(figsize=(8,5))
        if "linear" in results:
            plt.plot(t, results["linear"], label = "Linear system")
        if "nonlinear" in results:
            plt.plot(t, results["nonlinear"], "--", label = "Non-linear system")
        
        info = None
        if show_specs and "linear" in results:
            info = self.calculate_specs(results["linear"], t)

        if show_specs and info is not None:
            if "SteadyStateValue" in info:
                plt.axhline(info["SteadyStateValue"], linestyle=":", linewidth=1.2, label="Steady-State", color = "green")
            if "t_10" in info and info["t_10"] is not None:
                plt.axvline(info["t_10"], linestyle="--", linewidth=1.2, label=None, color = "black")
            if "t_90" in info and info["t_90"] is not None:
                plt.axvline(info["t_90"], linestyle="--", linewidth=1.2, label=None, color= "black")
            if "SettlingTime" in info and info["SettlingTime"] is not None:
                plt.axvline(info["SettlingTime"], linestyle="-.", linewidth=1.2, label="Settling time", color = "purple")
            
            if "t_10" in info and "t_90" in info and info["t_10"] is not None and info["t_90"] is not None:
                # pick a y-level (10% level works fine)
                y0 = results["linear"][0]
                y_ss = info["SteadyStateValue"]
                A = y_ss - y0
                y_10 = y0 + 0.10 * A

                # draw double-headed arrow
                plt.annotate("",
                            xy=(info["t_10"], y_10), xycoords="data",
                            xytext=(info["t_90"], y_10), textcoords="data",
                            arrowprops=dict(arrowstyle="<->", color="black"))

                # add label in the middle
                plt.text((info["t_10"] + info["t_90"]) / 2, y_10 - 0.05,
                        "Rise time", ha="center", va="top")
                
            # Overshoot arrow
            if "Peak" in info and "SteadyStateValue" in info:
                peak = info["Peak"]
                y_ss = info["SteadyStateValue"]
                t_peak = info["PeakTime"]

                if peak - y_ss > 1e-3:# Draw arrow from steady state to peak
                    plt.annotate("",
                                xy=(t_peak, y_ss), xycoords="data",
                                xytext=(t_peak, peak), textcoords="data",
                                arrowprops=dict(arrowstyle="<-", color="black"))

                    # Add label slightly to the right of arrow
                    plt.text(t_peak + 0.2, (peak + y_ss) / 2,
                            r"Overshoot $M_p$",
                            ha="left", va="center")

        plt.xlabel("Time [s]")
        plt.ylabel("Output y(t)")
        plt.title("System Response")
        plt.legend()
        plt.grid(True)
        plt.show()                       

In [39]:
def parse_matrix(text):
    """
    Valid example: "[[0,1],[-2,-3]]" -> np.array([[0,1],[-2,-3]])
    """
    try:
        return np.array(eval(text))
    except:
        return None

def parse_nonlinear(expr_str):
    """
    Valid example:
    "[x[1], -2*x[0] - 3*x[1] + u + 0.2*(x[0]**3)]"
    """
    def dyn(t, x, u_func):
        u_val = u_func(t) 
        return eval(expr_str, {"t": t, "x": x, "u": u_val, "np": np})
    return dyn



In [40]:
style_bold = {'description_width': '150px'}
layout_box = widgets.Layout(width="300px", height="70px")

A_box = widgets.Textarea(
    value="[[0, 1], [-2, -3]]",
    description="Matrix A:",
    layout=layout_box,
    style={'description_width': 'initial'}
)
A_box.layout = widgets.Layout(width="300px", height="70px")
A_box.add_class("bold-label")

B_box = widgets.Textarea(
    value="[[0], [1]]",
    description="Matrix B:",
    layout=layout_box,
    style={'description_width': 'initial'}
)

C_box = widgets.Textarea(
    value="[[1, 0]]",
    description="Matrix C:",
    layout=layout_box,
    style={'description_width': 'initial'}
)

D_box = widgets.Textarea(
    value="[[0]]",
    description="Matrix D:",
    layout=layout_box,
    style={'description_width': 'initial'}
)

nonlin_box = widgets.Textarea(
    value="[x[1], -2*x[0] - 3*x[1] + u + 0.2*(x[0]**3)]",
    description="Non-linear f(x,u,t):",
    layout=widgets.Layout(width="620px", height="70px"),
    style={'description_width': 'initial'}
)

eq_button = widgets.Button(
    description="Find equilibrium points",
    button_style="warning",
    layout=widgets.Layout(width="200px", height="40px")
)

eq_dropdown = widgets.Dropdown(
    options=[],
    description="Equilibrium points:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'}
)


input_box = widgets.ToggleButtons(
    options=["Step", "Impulse", "Sine", "Custom"],
    value="Step",
    description="Input:",
    button_style='info',
)

# sliders
T_box = widgets.FloatSlider(value=10, min=1, max=100, step=1, description="T:")

init_label = widgets.HTML(value="<span class='custom-title'>Initial conditions:</span>")
time_label = widgets.HTML(value="<span class='custom-title'>Time:</span>")

x0_0_label = widgets.HTML(value="<b>x<sub>1</sub>(0):</b>")
x0_0_box = widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, layout=widgets.Layout(width="300px"))

x0_1_label = widgets.HTML(value="<b>x<sub>2</sub>(0):</b>")
x0_1_box = widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, layout=widgets.Layout(width="300px"))

x0_0_ui = HBox([x0_0_label, x0_0_box])
x0_1_ui = HBox([x0_1_label, x0_1_box])

# button for non-linear
show_nonlin_box = widgets.ToggleButton(
    value=False,
    description="Show Non-linear",
    button_style='success',
    layout=widgets.Layout(width="200px", height="40px")
)

show_specs_box = widgets.ToggleButton(
    value = False, 
    description = "Show Time-Domain specifications",
    button_style='success',
    layout=widgets.Layout(width="265px", height="40px")

)

reset_button = widgets.Button(
    value=False,
    description="Reset",
    button_style='danger', 
    layout=widgets.Layout(width="200px", height="40px")
)

custom_input_box = widgets.Textarea(
    value="np.exp(-0.5*t)*np.cos(2*t)",
    description="Custom input u(t):",
    layout=widgets.Layout(width="620px", height="50px"),
    style={'description_width': 'initial'}
)

model_mode = widgets.ToggleButtons(
    options=["Matrices", "Linearized"],
    value="Matrices",
    description="Model:",
    button_style='',
    style={'description_width': 'initial'},
)

eq_row = HBox([eq_button, eq_dropdown])
eq_row.layout = widgets.Layout(margin="10px 0 0 0")  # add 10px margin on top


# final layout
ui = VBox([
    model_mode,
    HBox([A_box, B_box]),
    HBox([C_box, D_box]),
    nonlin_box,
    eq_row,
    input_box,
    custom_input_box,
    init_label,
    HBox([x0_0_ui, x0_1_ui]),
    time_label,
    T_box,
    HBox([show_nonlin_box, show_specs_box]),
    reset_button
])

def specs_available(y):
        if np.isnan(y).any() or np.isinf(y).any() or abs(y[-1]) > 1e6:
            return False  # divergent
        if np.std(y[-100:]) > 0.05 * abs(y[-1]):
            return False  # oscillatory
        return True

def _toggle_specs_visibility(input_value, results=None):
    if input_value == 'Step' and results is not None and "linear" in results and specs_available(results["linear"]):
        show_specs_box.layout.display = ''   # show
    else:
        show_specs_box.layout.display = 'none'  # hide

expr_list_cache = None

def compute_equilibria(b):
    global expr_list_cache
    try:
        # extract function ("[x[1], -2*x[0] - 3*x[1] + u + 0.2*(x[0]**3)]")
        expr_list = eval(nonlin_box.value, {"x": sp.symbols('x0:2'), "u": sp.symbols('u'), "np": np})
        f1, f2 = expr_list
        x0, x1, u = sp.symbols('x0 x1 u')
        f1 = sp.sympify(f1)
        f2 = sp.sympify(f2)
        
        expr_list_cache = [f1, f2]

        # find equilibrium points
        sols = sp.solve([sp.Eq(f1.subs(u, 0), 0), sp.Eq(f2.subs(u, 0), 0)], (x0, x1))
        if not sols:
            eq_dropdown.options = []
            print("Equilibrium point not found!")
        else:
            eq_dropdown.options = [(str(s), s) for s in sols]
    except Exception as e:
        print("Error:", e)

def linearize_system(expr_list, eq_point):
    x0, x1, u = sp.symbols('x0 x1 u')
    f = sp.Matrix(expr_list)

    A_sym = f.jacobian([x0, x1])
    B_sym = f.jacobian([u])

    subs_dict = {x0: eq_point[0], x1: eq_point[1], u: 0}
    A_num = np.array(A_sym.subs(subs_dict).evalf(), dtype=np.complex128)
    B_num = np.array(B_sym.subs(subs_dict).evalf(), dtype=np.complex128)

    C_num = np.array([[1, 0]])
    D_num = np.array([[0]])

    return A_num, B_num, C_num, D_num

In [41]:
out = Output()

def on_change(change=None):
    out.clear_output()
    with out:
        A = parse_matrix(A_box.value)
        B = parse_matrix(B_box.value)
        C = parse_matrix(C_box.value)
        D = parse_matrix(D_box.value)

        nonlinear_func = None
        if nonlin_box.value.strip() != "":
            nonlinear_func = parse_nonlinear(nonlin_box.value)

        sim = SystemSimulator(A, B, C, D, nonlinear_func)

        # choose which model to use depending on toggle
        if model_mode.value == "Matrices":
            # use manually entered matrices
            sim.sys_lin = StateSpace(A, B, C, D)

        elif model_mode.value == "Linearized":
            if expr_list_cache is not None and eq_dropdown.value not in (None, ''):
                A_lin, B_lin, C_lin, D_lin = linearize_system(expr_list_cache, eq_dropdown.value)
                display(Math(r"\textbf{Linearization at equilibrium point: }" + str(eq_dropdown.value)))
                sim.sys_lin = StateSpace(A_lin, B_lin, C_lin, D_lin)
            else:
                display(Math(r"\text{Warning: Please compute and select an equilibrium point to use linearized model.}"))
                sim.sys_lin = None


        # time and initial conditions
        t = np.linspace(0, T_box.value, 500)
        x0 = [x0_0_box.value, x0_1_box.value]

        # input
        if input_box.value == "Step":
            u_func = lambda t: 1.0 if t >= 0 else 0.0
        elif input_box.value == "Impulse":
            u_func = lambda t: 1.0 if abs(t) < 1e-2 else 0.0
        elif input_box.value == "Sine":
            u_func = lambda t: np.sin(2*np.pi*0.5*t)
        else:  # custom
            expr = custom_input_box.value
            def u_func(t):
                return eval(expr, {"t": t, "np": np})

        # simulation
        results = sim.simulate(t, x0, u_func)
        show_specs_flag = (input_box.value == "Step") and show_specs_box.value
        if not show_nonlin_box.value and "nonlinear" in results:
            results.pop("nonlinear")

        # update toggle visibility here
        _toggle_specs_visibility(input_box.value, results)    

        # plot
        sim.plot(t, results, show_specs=(input_box.value == "Step" and show_specs_box.value))

def reset_values(b):
    # base matrixes
    A_box.value = "[[0, 1], [-2, -3]]"
    B_box.value = "[[0], [1]]"
    C_box.value = "[[1, 0]]"
    D_box.value = "[[0]]"

    # nonlinear function
    nonlin_box.value = "[x[1], -2*x[0] - 3*x[1] + u + 0.2*(x[0]**3)]"

    # Input
    input_box.value = "Step"

    # initial conditions
    x0_0_box.value = 0.0
    x0_1_box.value = 0.0

    # time
    T_box.value = 10.0

    # Toggle nonlinear
    show_nonlin_box.value = False
    show_specs_box.value = False

    model_mode.value = "Matrices"
    
reset_button.on_click(reset_values)
eq_button.on_click(compute_equilibria)
eq_dropdown.observe(on_change, names="value")  

for w in [A_box, B_box, C_box, D_box, nonlin_box,
          input_box, custom_input_box, T_box, x0_0_box, x0_1_box, show_nonlin_box, show_specs_box, model_mode]:
    w.observe(on_change, names="value")  

# show everything together
display(ui, out)
on_change() 


Output()